# Example 1

In [1]:
path = '/home/justin/Desktop/spark/Ex_Files_Spark_ML_AI/Exercise Files/Ch03/03_02/clustering_dataset.csv'

In [2]:
df = spark.read.csv(path, header = True, inferSchema = True)

In [3]:
df.show()

+----+----+----+
|col1|col2|col3|
+----+----+----+
|   7|   4|   1|
|   7|   7|   9|
|   7|   9|   6|
|   1|   6|   5|
|   6|   7|   7|
|   7|   9|   4|
|   7|  10|   6|
|   7|   8|   2|
|   8|   3|   8|
|   4|  10|   5|
|   7|   4|   5|
|   7|   8|   4|
|   2|   5|   1|
|   2|   6|   2|
|   2|   3|   8|
|   3|   9|   1|
|   4|   2|   9|
|   1|   7|   1|
|   6|   2|   3|
|   4|   1|   9|
+----+----+----+
only showing top 20 rows



In [4]:
from pyspark.ml.feature import VectorAssembler

In [51]:
feature_df = VectorAssembler(inputCols = ['col1','col2','col3'], outputCol = 'features').transform(df)

In [52]:
feature_df.show()

+----+----+----+--------------+
|col1|col2|col3|      features|
+----+----+----+--------------+
|   7|   4|   1| [7.0,4.0,1.0]|
|   7|   7|   9| [7.0,7.0,9.0]|
|   7|   9|   6| [7.0,9.0,6.0]|
|   1|   6|   5| [1.0,6.0,5.0]|
|   6|   7|   7| [6.0,7.0,7.0]|
|   7|   9|   4| [7.0,9.0,4.0]|
|   7|  10|   6|[7.0,10.0,6.0]|
|   7|   8|   2| [7.0,8.0,2.0]|
|   8|   3|   8| [8.0,3.0,8.0]|
|   4|  10|   5|[4.0,10.0,5.0]|
|   7|   4|   5| [7.0,4.0,5.0]|
|   7|   8|   4| [7.0,8.0,4.0]|
|   2|   5|   1| [2.0,5.0,1.0]|
|   2|   6|   2| [2.0,6.0,2.0]|
|   2|   3|   8| [2.0,3.0,8.0]|
|   3|   9|   1| [3.0,9.0,1.0]|
|   4|   2|   9| [4.0,2.0,9.0]|
|   1|   7|   1| [1.0,7.0,1.0]|
|   6|   2|   3| [6.0,2.0,3.0]|
|   4|   1|   9| [4.0,1.0,9.0]|
+----+----+----+--------------+
only showing top 20 rows



In [53]:
from pyspark.ml.clustering import KMeans

In [87]:
kmeans = KMeans().setK(3)
#kmeans = kmeans.setSeed(2)

In [88]:
kmodel = kmeans.fit(feature_df)

In [89]:
kmodel.clusterCenters()

[array([80.        , 79.20833333, 78.29166667]),
 array([35.88461538, 31.46153846, 34.42307692]),
 array([5.12, 5.84, 4.84])]

In [96]:
output = kmodel.transform(feature_df)

In [97]:
output.show(80)

+----+----+----+-----------------+----------+
|col1|col2|col3|         features|prediction|
+----+----+----+-----------------+----------+
|   7|   4|   1|    [7.0,4.0,1.0]|         2|
|   7|   7|   9|    [7.0,7.0,9.0]|         2|
|   7|   9|   6|    [7.0,9.0,6.0]|         2|
|   1|   6|   5|    [1.0,6.0,5.0]|         2|
|   6|   7|   7|    [6.0,7.0,7.0]|         2|
|   7|   9|   4|    [7.0,9.0,4.0]|         2|
|   7|  10|   6|   [7.0,10.0,6.0]|         2|
|   7|   8|   2|    [7.0,8.0,2.0]|         2|
|   8|   3|   8|    [8.0,3.0,8.0]|         2|
|   4|  10|   5|   [4.0,10.0,5.0]|         2|
|   7|   4|   5|    [7.0,4.0,5.0]|         2|
|   7|   8|   4|    [7.0,8.0,4.0]|         2|
|   2|   5|   1|    [2.0,5.0,1.0]|         2|
|   2|   6|   2|    [2.0,6.0,2.0]|         2|
|   2|   3|   8|    [2.0,3.0,8.0]|         2|
|   3|   9|   1|    [3.0,9.0,1.0]|         2|
|   4|   2|   9|    [4.0,2.0,9.0]|         2|
|   1|   7|   1|    [1.0,7.0,1.0]|         2|
|   6|   2|   3|    [6.0,2.0,3.0]|

In [99]:
from pyspark.ml.linalg import Vectors

kmodel.predict(Vectors.dense([40.0,30.0,42.0]))

1

In [84]:
%%time
from pyspark.ml.evaluation import ClusteringEvaluator

silhouette_score = []

evaluator = ClusteringEvaluator(predictionCol = 'prediction', featuresCol = 'features', metricName = 'silhouette', distanceMeasure='squaredEuclidean')

for i in range(2,10):
    kmeans = KMeans().setK(i)
    kmodel = kmeans.fit(feature_df)
    output = kmodel.transform(feature_df)
    

    silhouette_score.append((i,evaluator.evaluate(output)))

CPU times: user 86.7 ms, sys: 9.65 ms, total: 96.3 ms
Wall time: 3.12 s


In [82]:
%%time
silhouette_score

CPU times: user 3 µs, sys: 1 µs, total: 4 µs
Wall time: 6.68 µs


[(2, 0.8467387149751704),
 (3, 0.8540025504534153),
 (4, 0.7480719929541821),
 (5, 0.7298593774153032),
 (6, 0.6172579772315678),
 (7, 0.6205845589347845),
 (8, 0.5680366469163016),
 (9, 0.6170296355060033)]

# Example 2